In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv, find_dotenv
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnablePassthrough

import bs4
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import TextLoader

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
_ = load_dotenv(find_dotenv())

groq_api_key = os.environ['GROQ_API_KEY']

In [3]:
llm = ChatGroq()

In [4]:
loader = TextLoader('./docS.txt', encoding='utf-8')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

vectorstore = Chroma.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever()

system_prompt = '''
'You are an assistant for question answering.
Use the following pieces of retrieved context to answer
{context}
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human','{input}')
    ]
)


D:\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
qa_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriever, qa_chain)

output = rag_chain.invoke({'input':'What are these articles about?'})

In [6]:
output['answer']

'These articles appear to be describing a fantasy world, featuring characters such as Zwinka, Wyndle, and a cesarz. The setting includes magical elements, such as a "bańka mocy" (sphere of power) surrounding Zwinka, and her ability to emit light and use "wspaniałość" (grace/beauty) to affect her surroundings. There are also mentions of magical creatures, like "wiatrospren" (small wind spirits) and "ostrołań" (large, thin, four-legged creatures). The characters seem to be on a journey through this world, encountering various obstacles such as trees, rocks, and fields of tall grass. Zwinka is established as different from the other characters because she is a "zdechlak" (dead thing), which in this context seems to mean that she is not affected by the passage of time and does not want to be tied down by relationships or familiar places. There is also a sense of tension or conflict, as Zwinka has left the cesarz without telling him and Wyndle is concerned about her well-being. Overall, the

In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [9]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is this article about?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What was my previous question about?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Your previous question was "What is this article about?"
User asked to provide the summary of the article and the context provided is from a fantasy story about a character named Zwinka who has some special abilities, she left a palace and being followed by a Pustkowec named Wyndle. Zwinka has some injury but she still moves forward and leaving a trail of uprooted trees and other destruction behind her.


In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [11]:
conversational_rag_chain.invoke(
    {"input": "What is this article about?"},
    config={
        "configurable": {"session_id": "001"}
    },  # constructs a key "001" in `store`.
)["answer"]

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


'This article appears to be a fantasy story about a character named Zwinka who has the ability to emit light and energy from her skin. She has left a palace and is traveling with a being named Wyndle, who is a Pustkowec, which may be a type of creature or being. Zwinka has the ability to topple trees and move quickly, and she seems to be resistant to being captured or constrained. She has a distant and independent personality, and she is wary of the people she encounters, believing that they may intend to harm her. The article includes descriptions of the landscape and other characters that Zwinka and Wyndle encounter on their journey. It is not clear what the overall plot or conflict of the story is.'

In [13]:
conversational_rag_chain.invoke(
    {"input": "What was my previous question about?"},
    config={"configurable": {"session_id": "001"}},
)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


{'input': 'What was my previous question about?',
 'chat_history': [HumanMessage(content='What is this article about?'),
  AIMessage(content='This article appears to be a fantasy story about a character named Zwinka who has the ability to emit light and energy from her skin. She has left a palace and is traveling with a being named Wyndle, who is a Pustkowec, which may be a type of creature or being. Zwinka has the ability to topple trees and move quickly, and she seems to be resistant to being captured or constrained. She has a distant and independent personality, and she is wary of the people she encounters, believing that they may intend to harm her. The article includes descriptions of the landscape and other characters that Zwinka and Wyndle encounter on their journey. It is not clear what the overall plot or conflict of the story is.'),
  HumanMessage(content='What was my previous question about?'),
  AIMessage(content='Your previous question asked for a summary of the content pr

In [25]:
for message in store["001"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is this article about?

AI: This article appears to be a description of a character named Zwinka who has special abilities, such as emitting light and causing objects to move without touching them. She is leaving a palace and is accompanied by a character named Wyndle. Zwinka is different from the other inhabitants of the palace, who are referred to as "Pustkowce" and "zdechlakami," and she prefers to be independent and not be limited to seeing the same things every day. The setting includes natural elements like sunlight, grass, trees, and a creature called a "chwałospren." The atmosphere is contemplative and slightly melancholic, with Wyndle expressing concern for Zwinka's well-being and reminiscing about her happy past in the palace. The style is poetic and imaginative, with vivid sensory language and metaphors. The topic of the article could be described as a portrait of a unique individual who seeks freedom and self-expression in a magical and mysterious world.

User: W